<a class="anchor" id="0."></a>
We will aplly classification algoritms on our data

# **Contents**

* [1. Import Libraries](#1.)
* [2. Analyze Data](#2.)
* [3. Data Preprocessing](#3.)
* [4. Classification Models](#4.)
* * [4.1. Logistic Regression](#4.1.)
* * [4.2. K-Nearest Neighbhour(KNN)](#4.2.)
* * [4.3. Suport Vector Machine(SVC)](#4.3.)
* * [4.4. Naive Bayes](#4.4.)
* * [4.5. Decision Tree](#4.5.)
* [5. Comparison of Models](#5.)
* [6. Conclusion](#6.)

[Go to the Head](#0.)

# <a class="anchor" id="1."></a>**1. Import Libraries**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
from plotly.offline import iplot
import plotly.graph_objs as go



# Close warning
import warnings
warnings.filterwarnings("ignore")



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# Any results you write to the current directory are saved as output.

Import Data

In [ ]:
data_2c = pd.read_csv('/kaggle/input/biomechanical-features-of-orthopedic-patients/column_2C_weka.csv')
data_3c = pd.read_csv('/kaggle/input/biomechanical-features-of-orthopedic-patients/column_3C_weka.csv')

[Go to the Head](#0.)

# <a class="anchor" id="2."></a>**2. Analyze Data**

Information about data

In [ ]:
data_2c.head()

In [ ]:
data_3c.info()

As we see there's not any null data. So we don't need modify data. Then let's analyze data.

In [ ]:
def look_univariate(dataset, discrete_feature):
    fig, plots=plt.subplots(nrows=1,ncols=2, figsize=(8,5))
    
    labels = dataset[discrete_feature].value_counts().index
    sizes = dataset[discrete_feature].value_counts().values
    
    dataset[discrete_feature].value_counts().plot(kind="bar",ax=plots[0])
    plt.pie(sizes,labels=labels,autopct="%1.1f%%") 
    plt.tight_layout()
    plt.show()

In [ ]:
look_univariate(data_3c,discrete_feature="class")

In [ ]:
look_univariate(data_2c,discrete_feature="class")

In [ ]:
data_norm = data_2c[data_2c["class"]=="Normal"]
data_abnorm = data_2c[data_2c["class"]=="Abnormal"]
data_2n = data_2c.drop(['class'],axis=1)

We can lok correlation map

In [ ]:
# correlation map
f,ax = plt.subplots(figsize=(10,10))
sns.heatmap(data_2n.corr(),annot=True,linewidths=.5,fmt='.1f',ax=ax)
plt.show()

In [ ]:
def class_wth_scatter(dataset):
    fig = ff.create_scatterplotmatrix(dataset,diag="box",colormap_type="cat",
                                      height=1000,width=1000)
    iplot(fig)

In [ ]:
class_wth_scatter(data_2n)

In [ ]:
sns.pairplot(data_3c, hue="class")
plt.show()

[Go to the Head](#0.)

# <a class="anchor" id="3."></a>**3. Data Preprocessing**

In [ ]:
data_2c['class'].value_counts()

In [ ]:
# change class data tpye
data_2c['class'] = [ 0 if each == "Abnormal" else 1 for each in data_2c['class']]
# x_data and y_data
x_data = data_2c.drop(["class"],axis=1)
y_data = data_2c["class"]

Firstly we need normalize data.

* Normalization Formula = (x - min(x))/(max(x) - min(x))|

In [ ]:
x = (x_data - np.min(x_data)) / (np.max(x_data) - np.min(x_data))

In [ ]:
x.head()

Before create model, we need separate data which is test and train.

In [ ]:
from sklearn.model_selection import train_test_split # import library for this
x_train,x_test,y_train,y_test = train_test_split(x_data,y_data,test_size = 0.3,random_state = 2)

print("x_train :", x_train.shape)
print("x_test :" , x_test.shape)
print("y_train :" , y_train.shape)
print("y_test :" , y_train.shape)

[Go to the Head](#0.)

# <a class="anchor" id="4."></a>**4. Classification Models**

In this kaggle we will do: 
* Logistic Regression Classification
* K-Nearest Neighbhour(KNN) Classification
* Support Vector Machine(SVM) Classification
* Naive Bayes Classification
* Decision Tree Classification
* Random Forest Classification
* Evaluate Classification all of these models
* Comparison of Models

[Go to the Head](#0.)

# <a class="anchor" id="4.1."></a>**4.1. Logistic Regression**

In [ ]:
# import library
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
# fit model
lr.fit(x_data,y_data)

lr_test_accuracy = lr.score(x_test,y_test)
lr_train_accuracy = lr.score(x_train,y_train)
lr_predict = lr.predict(x_test)

#Print Test and Train Accuracy
print("Test Accuracy(LR): ",lr.score(x_test,y_test))
print("Train Accuracy(LR): ",lr.score(x_train,y_train))

In [ ]:
# Confusion Matrix for Logistic Regression
from sklearn.metrics import confusion_matrix

y_true = y_test
y_predict = lr_predict

cm_lr = confusion_matrix(y_true,y_predict)

f,ax = plt.subplots(figsize = (8,6))
sns.heatmap(cm_lr,annot = True, linewidths=0.5,fmt = ".0f", cmap = "RdPu" , ax= ax)
plt.show()

[Go to the Head](#0.)

# <a class="anchor" id="4.2."></a>**4.2. K-Nearest Neighbhour(KNN)**

In [ ]:
# Implement KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3) # k value
knn.fit(x_train,y_train)
prediction = knn.predict(x_test)

#Print Test and Train Accuracy
print("KNN (K=3) test score is {}".format(knn.score(x_test,y_test)))
print("KNN (K=3) train score is {}".format(knn.score(x_train,y_train)))

So, it's correct k value? I don't know. We should try find optimal k value.

In [ ]:
# Optimal K value
neighbors = range(1,40)
knn_train = []
knn_test = []
for i,k in enumerate(neighbors):
    # k from 1 to 40 (40 exclude)
    knn_model = KNeighborsClassifier(n_neighbors=k)
    # fit knn
    knn_model.fit(x_train,y_train)
    knn_train.append(knn_model.score(x_train,y_train))    # train accuracy
    knn_test.append(knn_model.score(x_test,y_test))       # test accuracy
    

# vizualization results

# create trace1
trace1 = go.Scatter(
                    x = np.array(neighbors),
                    y = knn_train,
                    mode = "lines",
                    name="train accuracy",
                    marker = dict(color = 'rgba(160, 112, 2, 0.8)'),
                    text="train_accuracy")

# create trace2
trace2 = go.Scatter(
                    x = np.array(neighbors),
                    y = knn_test,
                    mode = "lines+markers",
                    name="test accuracy",
                    marker = dict(color = 'rgba(80, 26, 80, 0.8)'),
                    text="train_accuracy")

data = [trace1,trace2]

layout = dict(title = "K Value vs Accuracy",
             xaxis = dict(title = "Number of Neighbors", ticklen = 10,zeroline = True))

fig = dict(data = data, layout = layout)
iplot(fig)

knn_test_accuracy  = np.max(knn_test)
print("Best accuracy is {} with K = {}".format(np.max(knn_test),1 + knn_test.index(np.max(knn_test))))

In [ ]:
# Confusion Matrix for KNN
knn = KNeighborsClassifier(n_neighbors=10) # k value, we found K best value is 10
knn.fit(x_train,y_train)
y_predict = knn.predict(x_test)
y_true = y_test

from sklearn.metrics import confusion_matrix
cm_knn = confusion_matrix(y_true,y_predict)

f,ax = plt.subplots(figsize = (8,6))
sns.heatmap(cm_knn,annot=True,linewidths=0.5,fmt = ".0f", cmap = "RdPu" , ax= ax)
plt.show()

[Go to the Head](#0.)

# <a class="anchor" id="4.3."></a>**4.3. Support Vector Machine(SVM)**

In [ ]:
# import library
from sklearn.svm import SVC

svm = SVC(random_state=20)
svm.fit(x_train,y_train)

svm_test_accuray = svm.score(x_test,y_test)

print("SVM Test accucary: ",svm.score(x_test,y_test))
print("SVM Train accucary: ",svm.score(x_train,y_train))

In [ ]:
# Confusion Matrix for SVM
from sklearn.metrics import confusion_matrix
y_true = y_test
y_predict = svm.predict(x_test)

cm_svc = confusion_matrix(y_true,y_predict)

f,ax = plt.subplots(figsize = (8,6))
sns.heatmap(cm_svc,annot=True,linewidths=0.5,fmt=".0f",cmap="RdPu",ax = ax)
plt.show()

[Go to the Head](#0.)

# <a class="anchor" id="4.4."></a>**4.4. Naive Bayes**

In [ ]:
# import library
from sklearn.naive_bayes import GaussianNB

naive_bayes = GaussianNB()
naive_bayes.fit(x_train,y_train)

naive_bayes_score = naive_bayes.score(x_test,y_test)
nb_test_accuracy = naive_bayes.score(x_test,y_test)

print("naive bayes test score: ",naive_bayes.score(x_test,y_test))
print("naive bayes train score: ",naive_bayes.score(x_train,y_train))

In [ ]:
# Confusion Matrix for Naive Bayes
from sklearn.metrics import confusion_matrix

y_true = y_test
y_predict = naive_bayes.predict(x_test)

cm_nb = confusion_matrix(y_true,y_predict)

f,ax = plt.subplots(figsize = (8,6))
sns.heatmap(cm_nb,annot=True,linewidths=0.5,fmt=".0f",cmap="RdPu",ax = ax)
plt.show()

[Go to the Head](#0.)

# <a class="anchor" id="4.5."></a>**4.5. Decision Tree**

In [ ]:
# import library
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier()
dtree.fit(x_train,y_train)

decision_tree_score = dtree.score(x_test,y_test)
dt_test_accuracy = dtree.score(x_test,y_test)

print("decision tree test score: ",dtree.score(x_test,y_test))
print("decision tree train score: ",dtree.score(x_train,y_train))

In [ ]:
# Confusion Matrix for Decision Tree 
from sklearn.metrics import confusion_matrix

y_true = y_test
y_predict = dtree.predict(x_test)

cm_dt = confusion_matrix(y_true,y_predict)

f,ax = plt.subplots(figsize = (8,6))
sns.heatmap(cm_dt, annot=True,linewidths=0.5,fmt=".0f",cmap="RdPu",ax = ax)
plt.show()

[Go to the Head](#0.)

# <a class="anchor" id="4.5."></a>**4.6. Random Forest**

In [ ]:
# import library
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50,random_state=20)
rf.fit(x_train,y_train)

random_forest_score = rf.score(x_test,y_test)
rf_test_accuracy = rf.score(x_test,y_test)

print("random forest test score: ",rf.score(x_test,y_test))
print("random forest train score: ",rf.score(x_train,y_train))

In [ ]:
# Confusion Matrix for Random Forest
from sklearn.metrics import confusion_matrix

y_true = y_test
y_predict = rf.predict(x_test)

cm_rf = confusion_matrix(y_true,y_predict)

f,ax = plt.subplots(figsize = (8,6))
sns.heatmap(cm_rf,annot=True,linewidths=0.5,fmt=".0f",cmap="RdPu",ax = ax)
plt.show()

[Go to the Head](#0.)

# <a class="anchor" id="5."></a>**5. Comparison of Models**

In [ ]:
plt.figure(figsize=(20,10))
plt.suptitle("Confusion Matrices of Classification Models",fontsize=30)

plt.subplot(2,3,1)
plt.title("Logistic Regression Classification")
sns.heatmap(cm_lr,annot=True,cmap='YlGnBu',fmt=".0f",cbar=False)

plt.subplot(2,3,2)
plt.title("K Nearest Neighbors(KNN) Classification")
sns.heatmap(cm_knn,annot=True,cmap='YlGnBu',fmt=".0f",cbar=False)

plt.subplot(2,3,3)
plt.title("Support Vector Machine(SVM) Classification")
sns.heatmap(cm_svc,annot=True,cmap='YlGnBu',fmt=".0f",cbar=False)

plt.subplot(2,3,4)
plt.title("Naive Bayes Classification")
sns.heatmap(cm_nb,annot=True,cmap='YlGnBu',fmt=".0f",cbar=False)

plt.subplot(2,3,5)
plt.title("Decision Tree Classification")
sns.heatmap(cm_dt,annot=True,cmap='YlGnBu',fmt=".0f",cbar=False)

plt.subplot(2,3,6)
plt.title("Random Forest Classification")
sns.heatmap(cm_rf,annot=True,cmap='YlGnBu',fmt=".0f",cbar=False)

plt.show()

In [ ]:
acc_values = [lr_test_accuracy,knn_test_accuracy,svm_test_accuray,nb_test_accuracy,dt_test_accuracy,rf_test_accuracy]
model_names = ["logistic regression","KNN","SVM","naive bayes","decision tree","random forest"]
colors = ["green","red","blue","orange","yellow","brown"]

fig = go.Figure([go.Bar(x=model_names, y = acc_values,marker = dict(color = acc_values))])
fig.show()

[Go to the Head](#0.)
# <a class="anchor" id="6."></a>**6. Conclusion**

We did analyze data and implemented Classification models. If you like it, Please upvote my kernel. If you have any question, I will happy to hear it